In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/bangkok_traffy.csv')

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
#df.head()

In [4]:
footpath_keywords = [
    "ทางเท้า","ฟุตบาท","พื้นผิว","ฝาท่อ","แผ่นพื้น",
    "พื้นแตก","แตก","รอยแตก","หลุม","บ่อ",
    "พื้นทางเดิน","กระเบื้อง","แผ่นทางเท้า"
]

pattern = "|".join(footpath_keywords)

df_fp = df[
    df["type"].astype(str).str.contains(pattern, case=False, na=False) |
    df["comment"].astype(str).str.contains(pattern, case=False, na=False)
]

In [5]:
df_fp.shape

(210176, 16)

In [6]:
df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
df_fp["lon"] = pd.to_numeric(df_fp["lon"], errors="coerce")
df_fp["lat"] = pd.to_numeric(df_fp["lat"], errors="coerce")

/var/folders/pn/2yylzk651rq6ykx91h8t5jrm0000gn/T/ipykernel_33384/2938595523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
/var/folders/pn/2yylzk651rq6ykx91h8t5jrm0000gn/T/ipykernel_33384/2938595523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
/var/folders/pn/2yylzk651rq6ykx91h8t5jrm0000gn/T/ipykernel_33384/2938595523.py:2: SettingWithCopyWarning: 
A value is tryi

In [7]:
df_fp["timestamp"] = pd.to_datetime(df_fp["timestamp"], errors="coerce")
df_fp["last_activity"] = pd.to_datetime(df_fp["last_activity"], errors="coerce")

/var/folders/pn/2yylzk651rq6ykx91h8t5jrm0000gn/T/ipykernel_33384/300353576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp["timestamp"] = pd.to_datetime(df_fp["timestamp"], errors="coerce")
/var/folders/pn/2yylzk651rq6ykx91h8t5jrm0000gn/T/ipykernel_33384/300353576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp["last_activity"] = pd.to_datetime(df_fp["last_activity"], errors="coerce")


In [8]:
df_fp["duration_hours"] = (
    df_fp["last_activity"] - df_fp["timestamp"]
).dt.total_seconds() / 3600

df_fp["duration_hours"].describe()

/var/folders/pn/2yylzk651rq6ykx91h8t5jrm0000gn/T/ipykernel_33384/840720600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp["duration_hours"] = (


count    210176.000000
mean       1727.335198
std        3348.604418
min          -0.000399
25%          40.425008
50%         186.452099
75%        1577.000691
max       25121.953528
Name: duration_hours, dtype: float64

In [9]:
df_fp = df_fp[df_fp["duration_hours"] > 0]

df_fp = df_fp[df_fp["state"] =='เสร็จสิ้น']

lower_bound = df_fp["duration_hours"].quantile(0.01)
upper_bound = df_fp["duration_hours"].quantile(0.99)

df_fp["duration_hours"] = df_fp["duration_hours"].clip(lower=lower_bound, upper=upper_bound)
df_fp["duration_hours"].describe()

print(lower_bound)

1.34777204875


In [10]:
#df_fp.sort_values(by="duration_hours", ascending=True, inplace=True)
#df_fp.head()

In [11]:
df_fp.shape

(172931, 19)

In [13]:
output_path = "../data/processed/footpath_phase1.csv"
df_fp.to_csv(output_path, index=False)

len(df_fp), output_path

(172931, '../data/processed/footpath_phase1.csv')